In [2]:
import numpy as np

def relu(x):
    '''
    return output of Rectified Linear Unit and derivative
    '''
    f=np.maximum(0,x)
    df=np.where(x>0,1,0)
    return f,df
def sigmoid(x):

    f=1/(1+np.exp(-x))
    df=f*(1-f)
    return f,df
def tanh(x):
    f=-1+2/(1+np.exp(-2*x))
    df=1-f**2
    return f,df
def identity(x):
    '''
    恒等函数
    '''
    f=x
    df=1
    return f,df
def selu(x):
    alpha=1.67326
    lambd=1.05070
    f=lambd*np.where(x>=0,x,alpha*(np.exp(x)-1))
    df=lambd*np.where(x>=0,1,alpha*np.exp(x))
    return f,df
activation_table={
    'relu':relu,
    'sigmoid':sigmoid,
    'tanh':tanh,
    'identity':identity,
    'selu':selu
}
#loss function
def squared_error(y,yhat):
    '''
    input target predict
    return mean squared error
    '''
    return np.sum(0.5*(y-yhat)**2),y-yhat
def identity_loss(y,yhat):
    return yhat,yhat

loss_table={
    'squared_error':squared_error,
    'identity':identity_loss
}


In [84]:
class NeuralNetwork:
    def __init__(self,layer_dimensions,parameters):
        #init the neuralnetwork weights
        self.weights={}
        for i in range(len(layer_dimensions)-1):
            self.weights[i]=np.random.uniform(-0.1,0.1,
                                            (layer_dimensions[i],layer_dimensions[i+1]))
            
        self.learning_rate=parameters['learning_rate']
        self.num_iteration=parameters['num_iterations']
        self.batch_size=parameters['batch_size']

        activation_name=parameters['activation']
        if isinstance(activation_name,str) and activation_name in activation_table:
            self.activation=activation_table[activation_name]
        else:
            self.activation=activation_name
        loss_name=parameters['loss']
        if isinstance(loss_name,str) and loss_name in loss_table:
            self.loss=loss_table[loss_name]
        else:
            self.loss=loss_name
    
    def feedforward(self,x):
        #feed forward
        self.layer_input={}
        #the output of the previous layer and the input of the next layer
        self.layer_output={0:x}
        for i in range(len(self.weights)):
            self.layer_input[i]=np.dot(self.layer_output[i],self.weights[i])
            self.layer_output[i+1]=self.activation(self.layer_input[i])[0]
        return self.layer_output[len(self.weights)]
    #input is calculated by the inner product of the output of i and weight
    #output is calculated by activating the input 
    def backpropagation(self,y,yhat):
        num_layers=len(self.weights)
        #the output of last layer is predict
        delta=-1*self.loss(y,yhat)[1]*self.activation(self.layer_input[num_layers-1])[1]
        #initialize the last layer gradient weights
        gradient_weights={num_layers-1:np.dot(self.layer_output[num_layers-1].T,delta)}
        for i in reversed(range(num_layers-1)):
            delta=np.dot(delta,self.weights[i+1].T)*self.activation(self.layer_input[i])[1]
            gradient_weights[i]=np.dot(self.layer_output[i].T,delta)
        return gradient_weights
    
    def train(self,x_train,y_train):
        for iteration in range(self.num_iteration):
            if self.batch_size>0 and self.batch_size<x_train.shape[0]:
                batch_indices=np.random.choice(range(x_train.shape[0]),self.batch_size,replace=False)
                # print(x_train.iloc[[5,6]])
                x_batch=x_train.iloc[batch_indices]
                y_batch=y_train[batch_indices]
            else:
                x_batch=x_train
                y_batch=y_train
            # x_batch=x_train[:64][:]
            # y_batch=y_train[:64][:]
            y_hat=self.feedforward(x_batch)
            gradient_weights=self.backpropagation(y_batch,y_hat)
            for layer in range(len(self.weights)):
                self.weights[layer]-=self.learning_rate*gradient_weights[layer]
    def predict(self,x):
        activation_value=x
        for layer in range(len(self.weights)):
            weight_sum=np.dot(activation_value,self.weights[layer])
            activation_value=self.activation(weight_sum)[0]
        return activation_value 

In [4]:
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

mnist=fetch_openml("mnist_784",version=1)
x=mnist.data/255.0
y=mnist.target.astype(int)
encoder=LabelBinarizer()
y_one_hot=encoder.fit_transform(y)
x_train,x_test,y_train,y_test=train_test_split(x,y_one_hot,test_size=0.2,random_state=42)
print(x_train.shape,y_train.shape)

/home/adminroot/miniconda3/envs/pc_demo1/lib/python3.8/site-packages/sklearn/datasets/_openml.py:1002: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


(56000, 784) (56000, 10)


In [85]:
import numpy as np
layer_dimensions=[784,128,64,10]
parameters={
    'learning_rate':0.02,
    'num_iterations':2000,
    'batch_size':2,
    'activation':'tanh',
    'loss':'squared_error'
}
nn=NeuralNetwork(layer_dimensions,parameters)
nn.train(x_train,y_train)
y_pred=nn.predict(x_test)
correct_predictions=np.argmax(y_pred,axis=1)==np.argmax(y_test,axis=1)
accuarcy=np.mean(correct_predictions)
print(f"accuracy on test set:{accuarcy*100:.2f}%")

accuracy on test set:85.04%
